In [4]:
from llama_cpp import Llama
LLM_PATH = "../models/model-q8_0.gguf"

llm_model = Llama(
    model_path=LLM_PATH,
    n_gpu_layers=-1,
    n_batch=512,
    n_ctx=4096,
    n_parts=1,
    embedding=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32   

In [10]:
def aggregate_embeddings(embeddings: list) -> list:
    """
    Усредняет список эмбеддингов в один вектор.

    Параметры:
    - embeddings (list): Список эмбеддингов (списков чисел).

    Возвращает:
    list: Усовершенствованный вектор эмбеддингов.
    """
    # Усреднение по каждой координате
    aggregated_embedding = [sum(values) / len(values) for values in zip(*embeddings)]
    return aggregated_embedding

def get_text_embedding(model: Llama, text: str, normalize: bool = True) -> list:
    """
    Получает векторное представление (embedding) для заданного текста с использованием модели LLAMA.

    Параметры:
    - model (Llama): Инициализированная модель LLAMA.
    - text (str): Текст для получения векторного представления.
    - normalize (bool): Нормализовать ли векторное представление.

    Возвращает:
    list: Векторное представление текста.
    """
    # Получаем эмбеддинги для текста с помощью метода embed
    embeddings = model.embed(input=text, normalize=normalize, truncate=True)
    
    return embeddings

text = "Hello, how are you?"
embedding = get_text_embedding(llm_model, text)
print(len(embedding))

llama_perf_context_print:        load time =   12375.34 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     7 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2103.05 ms /     8 tokens


7


In [1]:
from llama_cpp import Llama

def get_text_embedding(model: Llama, text: str, normalize: bool = False) -> list:
    """
    Получает корректное векторное представление (эмбеддинг) для заданного текста
    с использованием модели LLAMA и агрегацией эмбеддингов.

    Параметры:
    - model (Llama): Инициализированная модель LLAMA.
    - text (str): Текст для получения векторного представления.
    - normalize (bool): Нормализовать ли векторное представление.

    Возвращает:
    list: Агрегированное векторное представление текста.
    """
    # Получение эмбеддингов для текста с помощью метода embed
    embeddings = model.embed(input=text, normalize=normalize)

    # Если embeddings - это не список, а единственный вектор (например, если текст короткий),
    # то возвращаем его напрямую
    if not isinstance(embeddings[0], list):
        return embeddings
    
    # Если embeddings - это список векторов, то агрегируем их методом усреднения
    aggregated_embedding = [sum(values) / len(values) for values in zip(*embeddings)]
    
    return aggregated_embedding

LLM_PATH = "../models/model-q8_0.gguf"

llm_model = Llama(
    model_path=LLM_PATH,
    n_gpu_layers=-1,
    n_batch=512,
    n_ctx=4096,
    n_parts=1,
    embedding=True
)

# Пример текста
text = "This is a sample text to check the embedding functionality."

# Получаем корректный эмбеддинг для текста
embedding = get_text_embedding(llm_model, text, normalize=True)
print("Корректный эмбеддинг текста:", embedding)
print("Длина эмбеддинга текста:", len(embedding))

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32   

Корректный эмбеддинг текста: [0.008135664632983883, -0.013391165497993587, 0.0030872178070310316, 0.0011327991153600686, 0.005147442235368338, -0.013490241199380927, 0.003888767139414147, -5.0454313438026004e-05, 0.003167268142928749, 0.005442210849973538, 0.00653016507164664, 0.00014695805362380178, -0.014140555635879333, -0.008319642753830372, -0.013535519802313144, -0.00042344320488094046, -0.009148545126878346, 0.005100843267016286, -0.012363734878620754, 0.006240413094640382, -0.006728326767302169, 0.0010729861050997468, -0.0013041561110388544, 0.007254549367221614, -0.012555713868246722, 0.008783514328084765, 0.007698096978554825, -0.012087120812049462, -0.0023918267812008506, 5.589557364105367e-05, -0.00523213233999173, 0.00025569128732630495, 0.0003410932872312743, 0.0029177306000895856, 0.0118379508134059, 0.002160664831251627, 0.004882942851434319, 0.006388674273483039, 0.006505730407651312, 0.002662130088994558, -0.0035882731052398753, -0.012369965974822109, 0.00808737976512

In [5]:
from typing import Any, List
from llama_cpp import Llama

MODEL_EMB_NAME = "ai-forever/sbert_large_nlu_ru"
SYSTEM_PROMPT = """
Эта система предназначена для предоставления полноценного анализа.

Инструкции:
1. Обработай входные данные и обеспечь правильное понимание контекста.
2. Проанализируй запрос пользователя, используя предоставленный контекст.
3. Предоставь четкий и полный анализ, относящийся к запросу пользователя.
4. Используй информацию из контекста для формирования ответа.

КОНТЕКСТ: {}
"""

SYSTEM_TOKEN = 1587
USER_TOKEN = 2188
BOT_TOKEN = 12435
LINEBREAK_TOKEN = 13

ROLE_TOKENS = {"user": USER_TOKEN, "bot": BOT_TOKEN, "system": SYSTEM_TOKEN}


def get_message_tokens(model: Llama, role: str, content: str) -> List[int]:
    """
    Создает токены для сообщения с учетом роли и содержания.

    Параметры:
    - model (Any): Модель токенизатора.
    - role (str): Роль сообщения.
    - content (str): Содержание сообщения.

    Возвращает:
    List[int]: Список токенов сообщения.

    Пример использования:
    ```python
    model = SomeTokenizer()
    role = "user"
    content = "Hello, world!"
    message_tokens = get_message_tokens(model, role, content)
    ```

    Подробности:
    - Функция токенизирует содержание сообщения с учетом роли и вставляет соответствующие токены.
    - В конце сообщения добавляется токен окончания строки.
    """
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def interact_manager(
    model: Llama,
    request: str,
    content: str,
    top_k: int = 30,
    top_p: float = 0.9,
    temperature: float = 0.6,
    repeat_penalty: float = 1.1,
):
    """
    Взаимодействие с моделью на основе LLAMA для генерации ответов на пользовательские запросы.

    Параметры:
    - model_path (str): Путь к предварительно обученной модели LLAMA.
    - user_prompt (str): Пользовательский запрос для генерации ответа.
    - n_ctx (int): Максимальная длина контекста.
    - top_k (int): Количество наиболее вероятных токенов для рассмотрения в генерации.
    - top_p (float): Порог отсечения для выбора токенов в генерации на основе вероятностей.
    - temperature (float): Параметр температуры для разнообразия в генерации.
    - repeat_penalty (float): Штраф за повторение токенов в генерации.

    Возвращает:
    str: Сгенерированный ответ на основе пользовательского запроса.

    Пример использования:
    ```python
    model_path = "path/to/model"
    user_prompt = "Привет, как дела?"
    response = interact(model_path, user_prompt)
    ```

    Подробности:
    - Функция использует модель LLAMA для генерации ответов на пользовательские запросы.
    - Задает параметры генерации, такие как ограничения токенов, температура и штраф за повторения.
    - Генерирует ответ на основе пользовательского запроса и возвращает его в виде строки.
    """
    tokens = []
    sys_prompt = SYSTEM_PROMPT.format(content)
    system_message = {"role": "system", "content": sys_prompt}
    tokens.extend(get_message_tokens(model, **system_message))
    # Получение токенов пользовательского сообщения
    message_tokens = get_message_tokens(
        model=model,
        role="user",
        content=request,
    )
    token_str = ""
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens

    # Генерация ответа на основе токенов
    generator = model.generate(
        tokens,
        top_k=top_k,
        top_p=top_p,
        temp=temperature,
        repeat_penalty=repeat_penalty,
    )

    # Преобразование токенов в строку
    for token in generator:
        token_str += model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)

        if token == model.token_eos():
            break

    return token_str

LLM_PATH = "../models/model-q8_0.gguf"

llm_model = Llama(
    model_path=LLM_PATH,
    n_gpu_layers=-1,
    n_batch=512,
    n_ctx=4096,
    n_parts=1,
)

interact_manager(llm_model, "Привет, как дела?", "Ответь что ты не знаешь ответа на данный вопрос")

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32   

''

In [3]:
from abc import ABC, abstractmethod
from typing import List, Dict, Optional
from llama_cpp import Llama
from rich.console import Console
from rich.table import Table
from pydantic import BaseModel, Field


class ModelKwargs(BaseModel):
    temperature: float | None = Field(default=0.7)
    top_k: int | None = Field(default=30)
    top_p: float | None = Field(default=0.9)
    max_tokens: int | None = Field(default=8192, ge=1, le=8192)
    repeat_penalty: float = Field(default=1.1)

class BaseModel(ABC):
    @abstractmethod
    def load_model(self, kwargs: ModelKwargs, model_path: str) -> None: ...

    @abstractmethod
    def inference(self, kwargs: Optional[ModelKwargs] = None) -> str: ...

    @abstractmethod
    def add_message(self, role: str, content: str) -> None: ...

    @abstractmethod
    def clean_chat(self) -> None: ...

    @abstractmethod
    def print_chat(self) -> None: ...

    @abstractmethod
    def get_chat(self) -> List[Dict[str, str]]: ...


class LLama3Quantized(BaseModel):
    def __init__(self) -> None:
        super().__init__()
        self._llm: Optional[Llama] = None
        self._chat: List[Dict[str, str]] = []
        self._kwargs: ModelKwargs = ModelKwargs()
        self._console = Console()

    def load_model(self, kwargs: ModelKwargs, model_path: str) -> None:
        if not model_path:
            raise ValueError("Путь к модели не может быть пустым.")
        self._llm = Llama(
            model_path=model_path,
            n_gpu_layers=-1,
            n_batch=512,
            n_ctx=kwargs.max_tokens,
            n_parts=1,
            embedding=True
        )
        self._kwargs = kwargs

    def _check_model_loaded(self) -> None:
        if not self._llm:
            raise RuntimeError(
                "Модель не загружена. Пожалуйста, загрузите модель перед выполнением вывода."
            )

    def inference(self, kwargs: Optional[ModelKwargs] = None) -> str:
        self._check_model_loaded()
        if not kwargs:
            kwargs = self._kwargs
        if not self._chat:
            raise RuntimeError(
                "Чат пуст. Пожалуйста, добавьте сообщения в чат перед выполнением вывода."
            )
        return self._llm.create_chat_completion(
            self._chat,
            temperature=kwargs.temperature,
            top_k=kwargs.top_k,
            top_p=kwargs.top_p,
            repeat_penalty=kwargs.repeat_penalty,
            stream=False,
        )["choices"][0]["message"]["content"]

    def add_message(self, role: str, content: str) -> None:
        if not role or not content:
            raise ValueError("Роль и содержание не могут быть пустыми.")
        self._chat.append({"role": role, "content": content})

    def clean_chat(self) -> None:
        self._chat = []

    def print_chat(self) -> None:
        if not self._chat:
            self._console.print("Чат пуст.", style="bold red")
            return
        table = Table(title="Чат", show_header=True, header_style="bold magenta")
        table.add_column("Роль", style="dim", width=12)
        table.add_column("Сообщение")
        for message in self._chat:
            for role, content in message.items():
                table.add_row(role.capitalize(), content)
        self._console.print(table)

    def get_chat(self) -> List[Dict[str, str]]:
        return self._chat
    
    def get_text_embedding(self, text: str, normalize: bool = False) -> list:
        """
        Получает корректное векторное представление (эмбеддинг) для заданного текста
        с использованием модели LLAMA и агрегацией эмбеддингов.

        Параметры:
        - model (Llama): Инициализированная модель LLAMA.
        - text (str): Текст для получения векторного представления.
        - normalize (bool): Нормализовать ли векторное представление.

        Возвращает:
        list: Агрегированное векторное представление текста.
        """
        # Получение эмбеддингов для текста с помощью метода embed
        embeddings = self._llm.embed(input=text, normalize=normalize)

        # Если embeddings - это не список, а единственный вектор (например, если текст короткий),
        # то возвращаем его напрямую
        if not isinstance(embeddings[0], list):
            return embeddings
        
        # Если embeddings - это список векторов, то агрегируем их методом усреднения
        aggregated_embedding = [sum(values) / len(values) for values in zip(*embeddings)]
        
        return aggregated_embedding


if __name__ == "__main__":
    llm = LLama3Quantized()
    kwargs = ModelKwargs(
        temperature=0.7,
        top_k=30,
        top_p=0.9,
        max_tokens=8192,
        repeat_penalty=1.1,
    )
    try:
        LLM_PATH = "../models/model-q8_0.gguf"
        llm.load_model(kwargs, LLM_PATH)
        print(llm.get_text_embedding("Ты Сайга и твоя задача помогать людям в решении вопросов."))
        llm.add_message(
            "system", "Ты Сайга и твоя задача помогать людям в решении вопросов."
        )
        llm.add_message("user", "Привет, в чём смысл вселенной?")
        answer = llm.inference()
        llm.add_message("assistant", answer)
        llm.add_message("user", "Спасибо за такой точный ответ!")
        answer2 = llm.inference()
        llm.print_chat()
        llm.clean_chat()
    except (ValueError, RuntimeError) as e:
        print(f"Ошибка: {e}")

ggml_metal_free: deallocating
llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.

[0.44052646841321674, 0.48467260297565234, 1.492329988451231, 0.6130397841334343, 1.0677956938743591, -0.5780240339892251, -1.4169881542523701, -0.1448425178061284, -2.4011075290895643, -0.674879776313901, -0.27452797832943143, 0.29581603762649356, 1.0689152990068709, -1.4575139326708657, -0.9086939380282447, 0.2643501128823984, -2.0856777646002316, -0.4381185487977096, -1.1039982308589278, 0.35944504255340215, -0.36951064460334326, -1.2693844519200779, 0.48489723673888613, -0.9184510729142598, 0.01204481011345273, -0.6556549160963013, 1.5075101724692754, 0.26668545816625866, 0.4127624744460696, 0.12962367279188974, -0.05359750915141333, 1.3978145236060733, 0.5298065337396803, -0.5744126151715007, 0.23881025505917414, 0.8247119738232522, -0.07004480764624618, -0.03305740672208014, 0.596741741611844, -0.018217044897721194, 0.6485543401823157, 0.37205548655419124, 1.5314192977689562, 0.6310021697343993, 2.3443254282077155, -0.5675737332730066, -0.3227978575797308, -0.2082099144213966, 0.

llama_get_logits_ith: invalid logits id -1, reason: no logits


: 